In [1]:
import datetime as dt
import os

import src.project_definitions as eb

import numpy as np
import pandas as pd

%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.gridspec import GridSpec
from matplotlib import ticker


from eblib import style, graphs
import eblib.utilities as ut
import eblib.sensor_reader as sr

In [2]:
##### USER INPUTS #####
ex = True
style.set_rc_eb_standard()


In [3]:
##### READ DATABASES #####

amb = sr.load_tf_weather(eb.files['tf']['WD']['1min'])

date_from = dt.date(2021,6,1)
#date_from = dt.date(2021,11,5)

date_to = dt.date.today()
#date_to = dt.date(2021,11,8)
delta_t=(date_to-date_from).days

In [4]:
#for bui in ['MW']:
for b, bui in enumerate(eb.buid):
    ind = sr.load_tf_bui(eb.files['tf'][bui]['1min'])
    ind = ind.loc[date_from:date_to]
    amb = amb.loc[date_from:date_to]
    for a, app in enumerate(eb.wohnungen):
    #for app in ['S']:
        for room in eb.rooms:
        #for room in ['K']:
            if room in ind[app].columns.get_level_values(level=0).unique():
                print(f'---plotting {eb.buid[bui]} | Wohnung {eb.wohnungen[app]} | {eb.rooms[room]} ---')
                #define figure
                fig = plt.figure(figsize=style.din_a3_landscape, constrained_layout=True)
                gs = GridSpec(10, 2, figure=fig)
                axs=[]
                i=0
                l=0
                fig.suptitle('Einfach Bauen Häuser - Bad Aibling\nAuswertung Messdaten\n{} | Wohnung {} | {}\n'.format(eb.buid[bui],eb.wohnungen[app],eb.rooms[room]), fontweight = 'bold')
                #----#1: Temperature ----
                sensor = 'trh_Tair'
                df = ind[app][room].filter(like=sensor)
                if len(df.columns) > 0:
                    if len(df.columns) > 1:
                        df.iloc[:,0].fillna(df.iloc[:,1],inplace=True)
                        df.drop(df.columns[1],axis=1,inplace=True)
                    df.columns = ['Tair']

                    Tair = df

                    TAMBG24 = amb['T_amb'].rolling('D').mean()
                    Tamb = amb['T_amb']

                    axs.append(fig.add_subplot(gs[l,:]))

                    axs[i].plot(df.resample('15min').last(),label = 'Raumlufttemperatur',alpha=0.75)
                    axs[i].plot(Tamb.resample('15min').last(),label = 'Außenlufttemperatur',alpha=0.75)

                    sensor = 'pt_Tsk'
                    df = ind[app][room].filter(like=sensor)
                    if len(df.columns) > 0:
                        axs[i].plot(df,label = 'Globethermometer',alpha=0.75)

                    ut.recolor_lines(axs[i], eb.cmap_temp)

                    axs[i].set_title('Temperaturen', fontweight = 'bold',loc='left')
                    axs[i].set_ylabel('Temperatur\n[°C]')
                    axs[i].legend(loc='center right',ncol=99,bbox_to_anchor=(1,1.1),frameon=False)
                    i+=1
                    l+=1
                else:
                    Tair = None
                    TAMBG24 = None
                    Tamb = None
                    #print('no {} sensor in room {}'.format(sensor,eb.rooms[room]))

                #----#2: Humidity ----
                sensor = 'trh_RH'
                df = ind[app][room].filter(like=sensor)
                if len(df.columns) > 0:
                    if len(df.columns) > 1:
                        df.iloc[:,0].fillna(df.iloc[:,1],inplace=True)
                        df.drop(df.columns[1],axis=1,inplace=True)
                    df.columns = ['Rh']

                    Rh = df.replace([' ',''],np.NAN)
                    Rh_amb = amb.Rh_amb

                    axs.append(fig.add_subplot(gs[l, :]))

                    axs[i].plot(Rh.resample('H').last(),label = 'Raumluftfeuchte',alpha=0.75)
                    axs[i].plot(Rh_amb.resample('H').last(),label = 'Außenluftfeuchte',alpha=0.75)
                    ut.recolor_lines(axs[i], eb.cmap_hum)

                    axs[i].set_title('Luftfeuchten', fontweight = 'bold',loc='left')
                    axs[i].set_ylabel('rel. Luftfeuchte\n[%]')
                    axs[i].set_ylim(0,100)
                    axs[i].legend(loc='center right',ncol=99,bbox_to_anchor=(1,1.1),frameon=False)
                    i+=1
                    l+=1
                else:
                    Rh=''
                    #print('no {} sensor in room {}'.format(sensor,eb.rooms[room]))

                #----#3: Heizkörpertemperatur ----
                sensor = 'pt_Thk'
                df = ind[app][room].filter(like=sensor)
                if len(df.columns) > 0:

                    axs.append(fig.add_subplot(gs[l,:]))

                    axs[i].plot(df.resample('H').last(),label = ut.get_labels(df),alpha=0.75)
                    ut.recolor_lines(axs[i], eb.cmap_temp)

                    axs[i].set_title('Heizkörpertemperaturen', fontweight = 'bold',loc='left')
                    axs[i].set_ylabel('Temperatur\n[°C]')
                    axs[i].set_ylim(0,55)
                    axs[i].legend(loc='center right',ncol=99,bbox_to_anchor=(1,1.1),frameon=False)
                    i+=1
                    l+=1
                else:
                    pass
                    #print('no {} sensor in room {}'.format(sensor,eb.rooms[room]))

                #----#4: Bewegungsmelder ----
                sensor = '_md'
                df = ind[app][room].filter(like=sensor)
                if len(df.columns) > 0:
                    df.columns = ['md']

                    axs.append(fig.add_subplot(gs[l,:]))

                    axs[i].plot(df.resample('H').sum(),label = 'Bewegungsmelder (stündliche Summe)',alpha=0.75)
                    axs[i].plot(df.resample('D').sum(),label = 'Bewegungsmelder (tägliche Summe)',alpha=0.75, ls='dashed')
                    ut.recolor_lines(axs[i], 'Purples_r' ,minint=0 ,maxint=0.8)

                    axs[i].set_title('Bewegungsmelder', fontweight = 'bold',loc='left')
                    axs[i].set_ylabel('Anwesenheit\n[min/h | min/T]')
                    axs[i].legend(loc='center right',ncol=99,bbox_to_anchor=(1,1.1),frameon=False)
                    i+=1
                    l+=1
                else:
                    pass
                    #print('no {} sensor in room {}'.format(sensor,eb.rooms[room]))

                #----#5: CO2 ----
                sensor = 'co2'
                if len(ind[app][room].filter(like=sensor).columns) == 0:
                    pass
                    #print('no {} sensor in room {}'.format(sensor,eb.rooms[room]))
                else: 
                    sensors = list(ind[app][room].filter(like=sensor).columns)
                    for sensor in sensors:
                        if sensor.count('_') == 1:
                            sensors = sensor

                    axs.append(fig.add_subplot(gs[l, :]))

                    axs[i].plot(ind[app][room][sensors].rolling('24H').mean(),label = 'CO2_mean (24h)',alpha=0.75,ls='dashed')
                    axs[i].plot(ind[app][room][sensors],label = 'CO2',alpha=0.75)

                    ut.recolor_lines(axs[i], eb.cmap_co2)

                    axs[i].set_title('CO2 Konzentration in der Raumluft', fontweight = 'bold',loc='left')
                    axs[i].set_ylabel('CO2 Konzentration\n[ppm]')
                    axs[i].legend(loc='center right',ncol=99,bbox_to_anchor=(1,1.1),frameon=False)
                    i+=1
                    l+=1

                #----#6: Windows ----
                windows = ind[app][room].filter(like='reed')
                if len(windows.columns) > 0:
                    axs.append(fig.add_subplot(gs[l, :]))
                    for window in windows.columns.str.split(' ',expand=True).droplevel(level=1).unique():
                        if len(windows.filter(like=window).columns) > 1:
                            windows.iloc[:,1] = windows.filter(like=window).iloc[:,1].fillna(windows.filter(like=window).iloc[:,0])
                            windows.drop(windows.filter(like=window).iloc[:,0].name,axis=1,inplace=True)
                    windows.replace(['Closed','Open'],[0,1],inplace=True)

                    axs[i].plot(windows.resample('D').sum(), label=ut.get_labels(windows),alpha=0.75, ls='dashed')
                    
                    axs.append(axs[i].twinx())

                    for w,col in enumerate(windows.columns):
                        norm = mpl.colors.Normalize(vmin=0, vmax=len(windows.columns))
                        cmap = mpl.cm.ScalarMappable(norm=norm, cmap=mpl.cm.Oranges)
                        cmap.set_array([])

                        axs[i+1].fill_between(x=windows.index, y1=windows[col], label=col,color=cmap.to_rgba(w + 1), alpha=0.1)

                    ut.recolor_lines(axs[i], 'Oranges_r', minint=0,maxint=0.5)
                    axs[i].set_title('Lüftungsverhalten', fontweight = 'bold',loc='left')
                    axs[i].set_ylabel('Fensteröffnung\n[min/d]')
                    han1,lab1 = axs[i].get_legend_handles_labels()
                    han2,lab2 = axs[i+1].get_legend_handles_labels()
                    han = han1 + han2
                    lab = lab1 + lab2
                    axs[i].legend(labels=lab,handles=han,loc='center right',ncol=99,bbox_to_anchor=(1,1.1),frameon=False)
                    #axs[i+1].legend(loc='upper center',ncol=3,frameon=False)
                    axs[i+1].set_ylim(0,2)
                    axs[i+1].set_yticks([0,1])
                    axs[i+1].set_yticklabels(['Geschlossen','Geöffnet'])
                    axs[i+1].tick_params(axis = 'y', which='major', length=0)
                    l+=1
                    i+=2
                else:
                    pass
                    #print('{}: Keine Fenster vorhanden'.format(room))

                for ax in axs:
                    if delta_t >= 30:
                        ax.xaxis.set_major_formatter(mdates.DateFormatter('%b %Y'))
                        ax.xaxis.set_major_locator(mdates.MonthLocator())                    
                    if delta_t < 30 and delta_t >= 5:
                        ax.xaxis.set_major_formatter(mdates.DateFormatter('%d.%m'))
                        ax.xaxis.set_major_locator(mdates.WeekdayLocator(0))
                    if delta_t < 5:
                        ax.xaxis.set_major_formatter(mdates.DateFormatter('%a %d.%m'))
                        ax.xaxis.set_minor_formatter(mdates.DateFormatter('%H:%M'))
                        ax.xaxis.set_minor_locator(mdates.HourLocator([8,16]))
                        ax.xaxis.set_major_locator(mdates.HourLocator([0]))
                    pass
                
                #----#7.1: Thermal Comfort ----
                if isinstance(Tair,(pd.Series, pd.DataFrame)) and isinstance(TAMBG24,(pd.Series, pd.DataFrame)):

                    axs.append(fig.add_subplot(gs[l:l+2, 0]))
                    graphs.thermal_comfort_2(
                        TAMBG24=TAMBG24.resample('H').last(),
                        TROOM=Tair.resample('H').last(),
                        ax=axs[i],
                        mode = 'air',
                        legend_ms=3
                        )
                    i+=1
                    #l+=2
                #----#7.2: H,x - Diagramm ----
                if isinstance(Tair,(pd.Series, pd.DataFrame)) and isinstance(Rh,(pd.Series, pd.DataFrame)):
                    axs.append(fig.add_subplot(gs[l:l+2, 1]))
                    graphs.comfort_hx_diagramm(
                        t1=Tair.resample('H').last(),
                        rh1=Rh.resample('H').last(),
                        rh2=Rh_amb.resample('H').last(),
                        t2=Tamb.resample('H').last(),
                        ax=axs[i],
                        legend_ms=3
                        )
                if ex:
                    ut.export(fig,filepath=os.path.join(eb.dir_results,'Übersicht',eb.buid[bui],eb.wohnungen[app]),filename='{}_{}_{}_Übersichtsgrafik'.format(bui,app,room), extlist=['pdf','png'],extsubfolder=True)
print('Done!')

C:\Users\Roman\AppData\Local\Temp/ipykernel_7528/3896306564.py:3: DtypeWarning: Columns (12,17,18,19,28,64,65,68,69,70,78,79,80,81,82,83,84,85,89,90,91,92,93,94,95,105) have mixed types.Specify dtype option on import or set low_memory=False.
  ind = sr.load_tf_bui(eb.files['tf'][bui]['1min'])


---plotting Massivholz | Wohnung Nord | Bad ---
C:\Users\Roman\workspace\eb-scripts\eb-data\Results\Übersicht\Massivholz\Nord\pdf\MH_N_B_Übersichtsgrafik.pdf saved!
C:\Users\Roman\workspace\eb-scripts\eb-data\Results\Übersicht\Massivholz\Nord\png\MH_N_B_Übersichtsgrafik.png saved!
---plotting Massivholz | Wohnung Nord | Flur ---
C:\Users\Roman\workspace\eb-scripts\eb-data\Results\Übersicht\Massivholz\Nord\pdf\MH_N_F_Übersichtsgrafik.pdf saved!
C:\Users\Roman\workspace\eb-scripts\eb-data\Results\Übersicht\Massivholz\Nord\png\MH_N_F_Übersichtsgrafik.png saved!
---plotting Massivholz | Wohnung Nord | Küche ---
C:\Users\Roman\workspace\eb-scripts\eb-data\Results\Übersicht\Massivholz\Nord\pdf\MH_N_K_Übersichtsgrafik.pdf saved!
C:\Users\Roman\workspace\eb-scripts\eb-data\Results\Übersicht\Massivholz\Nord\png\MH_N_K_Übersichtsgrafik.png saved!
---plotting Massivholz | Wohnung Nord | Schlafzimmer ---
C:\Users\Roman\workspace\eb-scripts\eb-data\Results\Übersicht\Massivholz\Nord\pdf\MH_N_SZ_Über

C:\Users\Roman\AppData\Local\Temp/ipykernel_7528/3896306564.py:3: DtypeWarning: Columns (7,13,14,19,20,29,63,69,70,73,76,77,78,84,94) have mixed types.Specify dtype option on import or set low_memory=False.
  ind = sr.load_tf_bui(eb.files['tf'][bui]['1min'])


---plotting Mauerwerk | Wohnung Nord | Bad ---
C:\Users\Roman\workspace\eb-scripts\eb-data\Results\Übersicht\Mauerwerk\Nord\pdf\MW_N_B_Übersichtsgrafik.pdf saved!
C:\Users\Roman\workspace\eb-scripts\eb-data\Results\Übersicht\Mauerwerk\Nord\png\MW_N_B_Übersichtsgrafik.png saved!
---plotting Mauerwerk | Wohnung Nord | Flur ---
C:\Users\Roman\workspace\eb-scripts\eb-data\Results\Übersicht\Mauerwerk\Nord\pdf\MW_N_F_Übersichtsgrafik.pdf saved!
C:\Users\Roman\workspace\eb-scripts\eb-data\Results\Übersicht\Mauerwerk\Nord\png\MW_N_F_Übersichtsgrafik.png saved!
---plotting Mauerwerk | Wohnung Nord | Küche ---
C:\Users\Roman\workspace\eb-scripts\eb-data\Results\Übersicht\Mauerwerk\Nord\pdf\MW_N_K_Übersichtsgrafik.pdf saved!
C:\Users\Roman\workspace\eb-scripts\eb-data\Results\Übersicht\Mauerwerk\Nord\png\MW_N_K_Übersichtsgrafik.png saved!
---plotting Mauerwerk | Wohnung Nord | Schlafzimmer ---
C:\Users\Roman\workspace\eb-scripts\eb-data\Results\Übersicht\Mauerwerk\Nord\pdf\MW_N_SZ_Übersichtsgrafi

C:\Users\Roman\AppData\Local\Temp/ipykernel_7528/3896306564.py:3: DtypeWarning: Columns (14,19) have mixed types.Specify dtype option on import or set low_memory=False.
  ind = sr.load_tf_bui(eb.files['tf'][bui]['1min'])


---plotting Leichtbeton | Wohnung Nord | Bad ---
C:\Users\Roman\workspace\eb-scripts\eb-data\Results\Übersicht\Leichtbeton\Nord\pdf\LB_N_B_Übersichtsgrafik.pdf saved!
C:\Users\Roman\workspace\eb-scripts\eb-data\Results\Übersicht\Leichtbeton\Nord\png\LB_N_B_Übersichtsgrafik.png saved!
---plotting Leichtbeton | Wohnung Nord | Flur ---
C:\Users\Roman\workspace\eb-scripts\eb-data\Results\Übersicht\Leichtbeton\Nord\pdf\LB_N_F_Übersichtsgrafik.pdf saved!
C:\Users\Roman\workspace\eb-scripts\eb-data\Results\Übersicht\Leichtbeton\Nord\png\LB_N_F_Übersichtsgrafik.png saved!
---plotting Leichtbeton | Wohnung Nord | Küche ---
C:\Users\Roman\workspace\eb-scripts\eb-data\Results\Übersicht\Leichtbeton\Nord\pdf\LB_N_K_Übersichtsgrafik.pdf saved!
C:\Users\Roman\workspace\eb-scripts\eb-data\Results\Übersicht\Leichtbeton\Nord\png\LB_N_K_Übersichtsgrafik.png saved!
---plotting Leichtbeton | Wohnung Nord | Schlafzimmer ---
C:\Users\Roman\workspace\eb-scripts\eb-data\Results\Übersicht\Leichtbeton\Nord\pdf\L

In [5]:
n_days = 30
n_weeks = 15

def timespan(df, days = 0, weeks = None, start=None, end=None):
    last_day = df.index.max().date()
    if isinstance(end,dt.date):
        last_day=end
    weekday = last_day.weekday()
    if isinstance(weeks, int):
        days = weeks*7
    start_day = last_day - dt.timedelta(days = days)
    if isinstance(start,dt.date):
        start_day = start
    df = df.loc[start_day:last_day]
    return df

pm = sr.load_tf_pm(eb.files['tf']['PM']['1min'])
amb = sr.load_tf_weather(eb.files['tf']['WD']['1min'])

if isinstance(n_days,int) or isinstance(n_weeks,int):
    pm = timespan(pm,days=n_days,weeks=n_weeks)
    amb = timespan(amb,days=n_days,weeks=n_weeks)

weekday = max(amb.index.max().date().weekday(),pm.index.max().date().weekday())
n_days = abs(max((amb.index.min()-amb.index.max()).days,(pm.index.min()-pm.index.max()).days))


fig = plt.figure(figsize=style.din_a4_landscape,constrained_layout=True)
fig.suptitle('Einfach Bauen Häuser - Bad Aibling\nAuswertung Wetterstation\n', fontweight = 'bold')
gs = GridSpec(5*2, 2, figure=fig)
axs=[]
i=0
l=0
#----#1: Temperaturen ----
axs.append(fig.add_subplot(gs[l,:]))
axs[i].set_title('Temperaturen', fontweight = 'bold',loc='left')
axs[i].set_ylabel('Temperatur\n[°C]')
axs[i].set_ylim(-20,40)
axs[i].plot(amb.T_amb,alpha=0.5,label='aktuelle Temperatur')
axs[i].plot(amb.T_amb.rolling('D').mean(), linestyle='dashed',label='Tagesmittel')
ut.recolor_lines(axs[i], eb.cmap_temp)
axs[i].legend(loc='center right',ncol=99,bbox_to_anchor=(1,1.1),frameon=False)
i+=1
l+=1
#----#2: Luftfeuchte ----
axs.append(fig.add_subplot(gs[l,:]))
axs[i].set_title('Luftfeuchte', fontweight = 'bold',loc='left')
axs[i].set_ylabel('rel. Luftfeuchte\n[%]')
axs[i].set_ylim(0,100)
axs[i].plot(amb.Rh_amb,alpha=0.5,label='aktuelle Luftfeuchte')
axs[i].plot(amb.Rh_amb.rolling('D').mean(), linestyle='dashed',label='Tagesmittel')
ut.recolor_lines(axs[i], 'Blues', minint=0.5,maxint=1)
axs[i].legend(loc='center right',ncol=99,bbox_to_anchor=(1,1.1),frameon=False)
i+=1
l+=1
#----#3: Niederschlag ----
axs.append(fig.add_subplot(gs[l,:]))
axs[i].set_title('Niederschlag', fontweight = 'bold',loc='left')
axs[i].set_ylabel('Niederschlag pro Minute\n[mm/min]')
axs[i].plot(amb.rain.diff(),alpha=0.5,label='aktueller Niederschlag')
han1,lab1 = axs[i].get_legend_handles_labels()
#axs[i].legend(loc='upper center',ncol=5,frameon=False)
axs.append(axs[i].twinx())
i+=1
axs[i].set_ylabel('Niederschlag pro Tag\n[mm/Tag]')
axs[i].plot(amb.rain.diff().rolling('D').sum(), linestyle='dashed',label='Niederschlag pro Tag')
han2,lab2 = axs[i].get_legend_handles_labels()
lab = lab1+lab2
han = han1+han2
axs[i].legend(labels = lab, handles = han, loc='center right',ncol=99,bbox_to_anchor=(1,1.1),frameon=False)
i+=1
l+=1
#----#3: Wind ----
axs.append(fig.add_subplot(gs[l,:]))
axs[i].set_title('Mittlere Windgeschwindigkeit pro Tag', fontweight = 'bold',loc='left')
axs[i].set_ylabel('Windgeschwindigkeit\n[m/s]')
axs[i].plot(amb.windspeed.rolling('D').mean(),alpha=1, label = 'Wind')  
axs[i].plot(amb.gustspeed.rolling('D').mean(),alpha=0.75,label = 'Bö',linestyle='dashed')  
ut.recolor_lines(axs[i], 'Greens_r', minint=0.3,maxint=0.7)
axs[i].legend(loc='center right',ncol=99,bbox_to_anchor=(1,1.1),frameon=False)
i+=1
l+=1
#----#4: Globalstrahlung ----
axs.append(fig.add_subplot(gs[l,:]))
axs[i].set_title('Mittlere Sonneneinstrahlung pro Tag', fontweight = 'bold',loc='left')
axs[i].set_ylabel('Strahlungsintensität\n[kWh/m²]')
axs[i].set_ylim(0,50)
df_p=pm.resample('H').sum().resample('D').mean()/1000

axs[i].bar(df_p.index,df_p['global'],label='global',color=plt.cm.Accent(0))
axs[i].bar(df_p.index,df_p['direct'],bottom = df_p['global'], label='direct',color=plt.cm.Accent(0.5))
axs[i].bar(df_p.index,df_p['diffuse'],bottom = df_p['direct'], label='diffuse',color=plt.cm.Accent(1))
axs[i].legend(loc='center right',ncol=99,bbox_to_anchor=(1,1.1),frameon=False)
y=amb.resample('H').last().isna().all(axis=1).replace([True,False],[1,0])

for ax in axs:
    if n_days > 60:
        ax.xaxis.set_major_locator(mdates.MonthLocator())
        ax.xaxis.set_major_formatter(mdates.DateFormatter('%b %Y'))
    if n_days <= 60:
        ax.xaxis.set_major_locator(mdates.WeekdayLocator(weekday))
        ax.xaxis.set_major_formatter(mdates.DateFormatter('%a, %d.%m'))

if ex:
    ut.export(fig,filepath=os.path.join(eb.dir_results,'Wetterstation'),filename='Wetterdaten', extlist=['pdf','png'])

C:\Users\Roman\workspace\eb-scripts\eb-data\Results\Wetterstation\Wetterdaten.pdf saved!
C:\Users\Roman\workspace\eb-scripts\eb-data\Results\Wetterstation\Wetterdaten.png saved!


In [6]:
winddir = {'E':4, 'ENE':3, 'ESE':5, 'N':0, 'NE':2, 'NNE':1, 'NNW':15, 'NW':14, 'S':8, 'SE':6, 'SSE':7,
       'SSW':9, 'SW':10, 'W':12, 'WNW':13, 'WSW':11}
winddir = dict(sorted(winddir.items(), key=lambda item: item[1]))
wind = dict(zip(winddir.keys(), np.linspace(0,2*np.pi,16,endpoint=False)))

W =  amb.loc[:,['winddir','windspeed','gustspeed']]
W['dir'] = W.winddir.replace(wind)

bins = np.linspace(W.windspeed.min(), W.windspeed.max(), 6)
labels = ['{} - {} m/s'.format(round(bins[i-1],1),round(bins[i],1)) for i in range(1,6)]

columns = ['bin{}'.format(i) for i in range(6)]
rose = pd.DataFrame(index=W.winddir.value_counts().index, columns=columns)

for i in range(1,6):
    rose[columns[i]] = W[(W.windspeed > bins[i-1]) & (W.windspeed <= bins[i])].groupby("winddir").size()/W.groupby('winddir').size().sum()
rose = rose*100

rose['theta'] = rose.index
rose.theta.replace(wind,inplace=True)
rose['deg'] =  rose.theta.apply(np.rad2deg)
rose.replace(np.nan,0,inplace=True)

fig, ax = plt.subplots(figsize = (6,6), subplot_kw={'projection': 'polar'})

fig.suptitle('Windrose',fontweight='bold')

w = 2*np.pi/(16)

_colors = plt.cm.Accent(np.linspace(0,1,5))

ax.bar(rose.theta,rose.bin1,width = w, bottom=0, label = labels[0],color=_colors[0],alpha=0.75)
ax.bar(rose.theta,rose.bin2,width = w, bottom=rose.bin1, label = labels[1],color=_colors[1],alpha=0.75)
ax.bar(rose.theta,rose.bin3,width = w, bottom=rose.bin1+rose.bin2, label = labels[2],color=_colors[2],alpha=0.75)
ax.bar(rose.theta,rose.bin4,width = w, bottom=rose.bin1+rose.bin2+rose.bin3, label = labels[3],color=_colors[3],alpha=0.75)
ax.bar(rose.theta,rose.bin5,width = w, bottom=rose.bin1+rose.bin2+rose.bin3+rose.bin4, label = labels[4],color=_colors[4],alpha=0.75)

ax.set_theta_direction(-1)

ax.set_xticks(np.linspace(0,2*np.pi,4,endpoint=False))
ax.set_xticks(np.linspace(0,2*np.pi,4,endpoint=False)+0.25*np.pi,minor=True)
ax.set_xticklabels(['NO','SO','SW','NW'],minor=True)
ax.set_xticklabels(['Nord','Ost','Süd','West'],fontweight='bold')

ax.set_theta_zero_location('N')

ax.grid(linestyle='dashed')
ax.grid(linestyle='dotted',which='minor')

ax.set_rlabel_position(60)
ax.legend(loc=3,frameon=True)
ax.yaxis.set_major_locator(ticker.FixedLocator([5,10,15,20]))
ax.yaxis.set_major_formatter(ticker.PercentFormatter(decimals=0))

fig.tight_layout()

if ex:
    ut.export(fig,filepath=os.path.join(eb.dir_results,'Wetterstation'),filename='Windrose', extlist=['pdf','png'])

C:\Users\Roman\workspace\eb-scripts\eb-data\Results\Wetterstation\Windrose.pdf saved!
C:\Users\Roman\workspace\eb-scripts\eb-data\Results\Wetterstation\Windrose.png saved!


In [7]:
for bui in eb.buid:
    ind=sr.load_tf_bui(eb.files['tf'][bui]['raw']).loc['2021-06-01':]
    fig = plt.figure(figsize=style.din_a4_landscape,constrained_layout=True)
    gs = GridSpec(3, 4, figure=fig)
    axs=[]

    fig.suptitle('{}'.format(eb.buid[bui]),fontweight='bold')
    for i, app in enumerate(eb.wohnungen):
        axs.append([])
        dfs = []

        for room in eb.rooms:
            try:
                idx = []
                df = ind[app][room].filter(like='reed')                    
                for window in df.columns.str.split(' ',expand=True).droplevel(level=1).unique():
                    if len(df.filter(like=window).columns) > 1:
                        df.iloc[:,1] = df.filter(like=window).iloc[:,1].fillna(df.filter(like=window).iloc[:,0])
                        df.drop(df.filter(like=window).iloc[:,0].name,axis=1,inplace=True)
                df.replace(['Closed','Open'],[0,1],inplace=True)
                for f,col in enumerate(df.columns):
                    t = col.split(' ')
                    ori = t[0].split('_')[0]
                    idx.append((room,'{}{}_{}'.format(ori,f, t[1])))
                df.columns = pd.MultiIndex.from_tuples(idx)
                dfs.append(df)
            except:
                continue

        df = pd.concat(dfs).resample('1min').max()

        n=len(df.columns)
        df1 = df.sum(axis=1)
        df1 = df1[df1 > 0]
        counts = df1.value_counts(normalize=True)
        gl = round(df1.dropna().mean(),1)
        ventkoeff = round(df.max(axis=1).value_counts(normalize=True)[1.0]*100,1)    #Anteil der Zeit in der Fenster geöffnet sind. Gewichtet nach Anzahl der Fenster.
        
        ### Histogramm
        j=0
        axs[i].append(fig.add_subplot(gs[i, 0]))
        df3 = df.sum(axis=1)
        df3.hist(ax = axs[i][j],bins=np.linspace(0,n+1,n+2), grid = False)
        axs[i][j].set_xticks(np.linspace(0,n,n+1)+0.5)
        axs[i][j].set_xticklabels(range(n+1))
        ut.ticks_to_hours(axs[i][j])
        axs[i][j].set_ylabel('Zeit [h]')
        axs[i][j].set_title('Häufigkeitsverteilung ')
        ###Timeline
        j+=1
        axs[i].append(fig.add_subplot(gs[i, 1:]))
        axs[i][j].set_title('Wohnung {}'.format(eb.wohnungen[app]),fontweight='bold')
        axs[i][j].text(
                0.0,
                0.95, 
                'Fenster pro Wohnung: {}\nZeit mit min. einem geöffnetem Fenster: {} %\nDurschnittlich gleichzeitig geöffnete Fenster: {}\nMaximal gleichzeitig geöffnete Fenster: {}'.format(n,ventkoeff,gl,int(df1.max())),      
                fontsize = 'large', 
                style='normal', 
                ha = 'left', 
                va = 'top',
                transform=axs[i][j].transAxes,
                bbox=style.eb_bbox, 
                )
        axs[i][j].set_ylim(0,24*60*n)
        axs[i][j].set_ylabel('Zeit\n' + r'$[n_{Fenster}*min_{geöffnet}]$')
        df2 = df.sum(axis=1).resample('D').sum()
        axs[i][j].bar(df2.index, df2)
        axs[i][j].plot(df2.rolling('30D').mean(),'r--')

    for ax in axs:
        ax[1].xaxis.set_major_locator(mdates.MonthLocator())
        ax[1].xaxis.set_major_formatter(mdates.DateFormatter('%b %Y'))
    if ex:
        ut.export(fig,filepath=os.path.join(eb.dir_results,'Lüftungsverhalten'),filename=f'{bui}_Übersicht', extlist=['pdf','png'])

C:\Users\Roman\AppData\Local\Temp/ipykernel_7528/3015283809.py:2: DtypeWarning: Columns (12,17,18,19,28,64,65,68,69,70,78,79,80,81,82,83,84,85,89,90,91,92,93,94,95,105) have mixed types.Specify dtype option on import or set low_memory=False.
  ind=sr.load_tf_bui(eb.files['tf'][bui]['raw']).loc['2021-06-01':]
C:\Users\Roman\anaconda3\envs\einfach-bauen\lib\site-packages\eblib\utilities.py:31: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_yticklabels(new_ticks)
C:\Users\Roman\anaconda3\envs\einfach-bauen\lib\site-packages\eblib\utilities.py:31: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_yticklabels(new_ticks)
C:\Users\Roman\anaconda3\envs\einfach-bauen\lib\site-packages\eblib\utilities.py:31: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_yticklabels(new_ticks)


C:\Users\Roman\workspace\eb-scripts\eb-data\Results\Lüftungsverhalten\MH_Übersicht.pdf saved!
C:\Users\Roman\workspace\eb-scripts\eb-data\Results\Lüftungsverhalten\MH_Übersicht.png saved!


C:\Users\Roman\AppData\Local\Temp/ipykernel_7528/3015283809.py:2: DtypeWarning: Columns (14,19,84,94) have mixed types.Specify dtype option on import or set low_memory=False.
  ind=sr.load_tf_bui(eb.files['tf'][bui]['raw']).loc['2021-06-01':]
C:\Users\Roman\anaconda3\envs\einfach-bauen\lib\site-packages\eblib\utilities.py:31: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_yticklabels(new_ticks)
C:\Users\Roman\anaconda3\envs\einfach-bauen\lib\site-packages\eblib\utilities.py:31: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_yticklabels(new_ticks)
C:\Users\Roman\anaconda3\envs\einfach-bauen\lib\site-packages\eblib\utilities.py:31: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_yticklabels(new_ticks)


C:\Users\Roman\workspace\eb-scripts\eb-data\Results\Lüftungsverhalten\MW_Übersicht.pdf saved!
C:\Users\Roman\workspace\eb-scripts\eb-data\Results\Lüftungsverhalten\MW_Übersicht.png saved!


C:\Users\Roman\AppData\Local\Temp/ipykernel_7528/3015283809.py:2: DtypeWarning: Columns (14,19) have mixed types.Specify dtype option on import or set low_memory=False.
  ind=sr.load_tf_bui(eb.files['tf'][bui]['raw']).loc['2021-06-01':]
C:\Users\Roman\anaconda3\envs\einfach-bauen\lib\site-packages\eblib\utilities.py:31: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_yticklabels(new_ticks)
C:\Users\Roman\anaconda3\envs\einfach-bauen\lib\site-packages\eblib\utilities.py:31: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_yticklabels(new_ticks)
C:\Users\Roman\anaconda3\envs\einfach-bauen\lib\site-packages\eblib\utilities.py:31: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_yticklabels(new_ticks)


C:\Users\Roman\workspace\eb-scripts\eb-data\Results\Lüftungsverhalten\LB_Übersicht.pdf saved!
C:\Users\Roman\workspace\eb-scripts\eb-data\Results\Lüftungsverhalten\LB_Übersicht.png saved!
